In [1]:

import sys
sys.path.append('src/')
from src.logic_utils import get_lang

import torch
import numpy as np

In [2]:
lark_path = 'src/lark/exp.lark'
lang_base_path = 'data/lang/'
lang, clauses, bk, atoms = get_lang(
        lark_path, lang_base_path,'coinjump', 'coinjump')

In [3]:
atoms

[.(__F__),
 .(__T__),
 closeby(obj1,obj2),
 closeby(obj1,obj3),
 closeby(obj1,obj4),
 closeby(obj2,obj1),
 closeby(obj2,obj3),
 closeby(obj2,obj4),
 closeby(obj3,obj1),
 closeby(obj3,obj2),
 closeby(obj3,obj4),
 closeby(obj4,obj1),
 closeby(obj4,obj2),
 closeby(obj4,obj3),
 have_key(img),
 jump(obj1),
 jump(obj2),
 jump(obj3),
 jump(obj4),
 left_go_get_key(obj1),
 left_go_get_key(obj2),
 left_go_get_key(obj3),
 left_go_get_key(obj4),
 left_go_to_door(obj1),
 left_go_to_door(obj2),
 left_go_to_door(obj3),
 left_go_to_door(obj4),
 not_have_key(img),
 on_left(obj1,obj2),
 on_left(obj1,obj3),
 on_left(obj1,obj4),
 on_left(obj2,obj1),
 on_left(obj2,obj3),
 on_left(obj2,obj4),
 on_left(obj3,obj1),
 on_left(obj3,obj2),
 on_left(obj3,obj4),
 on_left(obj4,obj1),
 on_left(obj4,obj2),
 on_left(obj4,obj3),
 on_right(obj1,obj2),
 on_right(obj1,obj3),
 on_right(obj1,obj4),
 on_right(obj2,obj1),
 on_right(obj2,obj3),
 on_right(obj2,obj4),
 on_right(obj3,obj1),
 on_right(obj3,obj2),
 on_right(obj3,obj

In [4]:

from percept import SlotAttentionPerceptionModule, YOLOPerceptionModule
from facts_converter import FactsConverter
from nsfr import NSFReasoner
from logic_utils import build_infer_module, generate_atoms
from valuation import RLValuationModule
device = torch.device('cpu')

In [5]:
def get_nsfr_model(lang, clauses, atoms, bk, device):
    PM = YOLOPerceptionModule(e=4, d=11, device=device)
    VM = RLValuationModule(
            lang=lang, device=device)
    FC = FactsConverter(lang=lang, perception_module=PM,
                        valuation_module=VM, device=device)
    IM = build_infer_module(clauses, atoms, lang,
                            m=len(clauses), infer_step=4, device=device)
    # Neuro-Symbolic Forward Reasoner
    NSFR = NSFReasoner(perception_module=PM, facts_converter=FC,
                       infer_module=IM, atoms=atoms, bk=bk, clauses=clauses)
    return NSFR

In [6]:
NSFR = get_nsfr_model(lang, clauses, atoms, bk, device=device)

Loading YOLO model...


In [7]:
metric =[[1,0,0,0,11,2],[0,1,0,0,14,2],[0,0,1,0,14.1,2],[0,0,0,1,20,2]]
x = torch.tensor(np.array(metric), dtype=torch.float32).unsqueeze(0)

In [8]:
x.shape

torch.Size([1, 4, 6])

In [9]:
V_T = NSFR(x)

In [10]:
NSFR.print_valuation_batch(V_T)

====== LEARNED PROGRAM ======
C_0:  jump(O1):-type(O1,agent),type(O2,enemy),closeby(O1,O2). 1.0
C_1:  left_go_get_key(01):-type(01,agent),type(02,key),on_left(02,01),not_have_key(X). 1.0
C_2:  right_go_get_key(01):-type(01,agent),type(02,key),on_right(02,01),not_have_key(X). 1.0
C_3:  left_go_to_door(01):-type(01,agent),type(02,door),on_left(02,01),have_key(X). 1.0
C_4:  right_go_to_door(01):-type(01,agent),type(02,door),on_right(02,01),have_key(X). 1.0
===== BATCH:  0 =====
37 on_left(obj4,obj1) :  1.0
60 type(obj1,agent) :  1.0
34 on_left(obj3,obj1) :  1.0
35 on_left(obj3,obj2) :  1.0
27 not_have_key(img) :  1.0
74 type(obj4,enemy) :  1.0
45 on_right(obj2,obj4) :  1.0
39 on_left(obj4,obj3) :  1.0
67 type(obj2,key) :  1.0
38 on_left(obj4,obj2) :  1.0
40 on_right(obj1,obj2) :  1.0
69 type(obj3,door) :  1.0
41 on_right(obj1,obj3) :  1.0
42 on_right(obj1,obj4) :  1.0
48 on_right(obj3,obj4) :  1.0
44 on_right(obj2,obj3) :  1.0
1 .(__T__) :  1.0
31 on_left(obj2,obj1) :  1.0
6 closeby(obj2,

In [11]:
V_T

tensor([[0.04395, 1.00000, 0.04395, 0.04395, 0.04395, 0.04395, 0.99623, 0.04395, 0.04395, 0.99623, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.05310, 0.04405, 0.04405, 0.04400, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 1.00000, 0.04395, 0.04395, 0.04395, 1.00000, 0.04395, 0.04395,
         1.00000, 1.00000, 0.04395, 1.00000, 1.00000, 1.00000, 1.00000, 1.00000, 1.00000, 0.04395, 1.00000, 1.00000, 0.04395, 0.04395, 1.00000, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 1.00000, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 0.04395, 1.00000,
         0.04395, 1.00000, 0.04395, 0.04395, 0.04395, 0.04395, 1.00000, 0.04395]])

In [12]:
atoms

[.(__F__),
 .(__T__),
 closeby(obj1,obj2),
 closeby(obj1,obj3),
 closeby(obj1,obj4),
 closeby(obj2,obj1),
 closeby(obj2,obj3),
 closeby(obj2,obj4),
 closeby(obj3,obj1),
 closeby(obj3,obj2),
 closeby(obj3,obj4),
 closeby(obj4,obj1),
 closeby(obj4,obj2),
 closeby(obj4,obj3),
 have_key(img),
 jump(obj1),
 jump(obj2),
 jump(obj3),
 jump(obj4),
 left_go_get_key(obj1),
 left_go_get_key(obj2),
 left_go_get_key(obj3),
 left_go_get_key(obj4),
 left_go_to_door(obj1),
 left_go_to_door(obj2),
 left_go_to_door(obj3),
 left_go_to_door(obj4),
 not_have_key(img),
 on_left(obj1,obj2),
 on_left(obj1,obj3),
 on_left(obj1,obj4),
 on_left(obj2,obj1),
 on_left(obj2,obj3),
 on_left(obj2,obj4),
 on_left(obj3,obj1),
 on_left(obj3,obj2),
 on_left(obj3,obj4),
 on_left(obj4,obj1),
 on_left(obj4,obj2),
 on_left(obj4,obj3),
 on_right(obj1,obj2),
 on_right(obj1,obj3),
 on_right(obj1,obj4),
 on_right(obj2,obj1),
 on_right(obj2,obj3),
 on_right(obj2,obj4),
 on_right(obj3,obj1),
 on_right(obj3,obj2),
 on_right(obj3,obj

In [13]:
for i, atom in enumerate(atoms):
    print(V_T[0][i], atom)

tensor(0.04395) .(__F__)
tensor(1.) .(__T__)
tensor(0.04395) closeby(obj1,obj2)
tensor(0.04395) closeby(obj1,obj3)
tensor(0.04395) closeby(obj1,obj4)
tensor(0.04395) closeby(obj2,obj1)
tensor(0.99623) closeby(obj2,obj3)
tensor(0.04395) closeby(obj2,obj4)
tensor(0.04395) closeby(obj3,obj1)
tensor(0.99623) closeby(obj3,obj2)
tensor(0.04395) closeby(obj3,obj4)
tensor(0.04395) closeby(obj4,obj1)
tensor(0.04395) closeby(obj4,obj2)
tensor(0.04395) closeby(obj4,obj3)
tensor(0.04395) have_key(img)
tensor(0.05310) jump(obj1)
tensor(0.04405) jump(obj2)
tensor(0.04405) jump(obj3)
tensor(0.04400) jump(obj4)
tensor(0.04395) left_go_get_key(obj1)
tensor(0.04395) left_go_get_key(obj2)
tensor(0.04395) left_go_get_key(obj3)
tensor(0.04395) left_go_get_key(obj4)
tensor(0.04395) left_go_to_door(obj1)
tensor(0.04395) left_go_to_door(obj2)
tensor(0.04395) left_go_to_door(obj3)
tensor(0.04395) left_go_to_door(obj4)
tensor(1.) not_have_key(img)
tensor(0.04395) on_left(obj1,obj2)
tensor(0.04395) on_left(obj1,